In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import sys; sys.path.append("../src")

In [ ]:
from models.cgans import AirfoilAoACEGAN, AirfoilAoAGenerator
from train_final_cebgan import read_configs, assemble_new_gan
from utils.dataloader import AirfoilDataset, NoiseGenerator

In [ ]:
from torch.utils.data import DataLoader

# Load Trained CEBGAN

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_dir = '../saves/final/'
checkpoint = 'cebgan14999.tar'
base_dir = "../src/"
dis_cfg, gen_cfg, egan_cfg, cz, noise_type = read_configs('cebgan', base_dir=base_dir)

In [ ]:
batch = 4096

In [ ]:
egan = assemble_new_gan(dis_cfg, gen_cfg, egan_cfg, device=device)
egan.load(os.path.join(save_dir, checkpoint), train_mode=False)

In [ ]:
### LOAD TEST DATA ###
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

inp_paras = np.load('../data/inp_paras_995.npy').astype(np.float32)
mean_std = (inp_paras.mean(0), inp_paras.std(0))

In [ ]:
noise_gen = NoiseGenerator(batch, sizes=cz, device=device, noise_type=noise_type, output_prob=True)

# Test SLL on Predictions
Correlation between Surrogate Log Likelihood and Airfoil Performance
## Surrogate Log Likelihood

In [ ]:
inp_paras_org = np.load('../data/inp_paras_347.npy')
inp_paras_250 = np.load('../data/inp_paras_250.npy')

In [ ]:
index = []
for each in inp_paras_250:
    index.append(np.where((inp_paras_org == each).prod(-1)))
index = np.concatenate(index).squeeze()
print(index.shape)

In [ ]:
airfoils_pred = np.load('../data/pred_cebgan/batch/new/airfoils_pred.npy')[index]
aoas_pred = np.load('../data/pred_cebgan/batch/new/aoas_pred.npy')[index]
inp_paras_pred = np.repeat(np.expand_dims(np.load('../data/inp_paras_250.npy'), 1), 10, axis=1)

In [ ]:
predictions = AirfoilDataset(
    inp_paras_pred.reshape(-1, 3),
    airfoils_pred.reshape(-1, 192, 2),
    aoas_pred.reshape(-1, 1),
    inp_mean_std=mean_std, device=device)
pred_dataloader = DataLoader(predictions, batch_size=1, shuffle=False)

In [ ]:
ll = egan.surrogate_ll(pred_dataloader, noise_gen).cpu().detach().numpy()

## Performance Data

In [ ]:
ll_pred = ll.reshape(250, -1)

In [ ]:
rs_s = [np.load('../results/efys_vad_cebgan_mul{}.npy'.format(i+1)) for i in range(10)]
rs = np.hstack(rs_s)

In [ ]:
plt.hist(ll_pred.reshape(-1), bins=30)
plt.show()

In [ ]:
plt.hist(rs.reshape(-1), bins=20, range=[0, 2000])
plt.show()

## Evaluate Correlation Coefficient

In [ ]:
cre = []
for l, r in zip(ll_pred, rs):
    cre.append(np.corrcoef(l[r>0], r[r>0])[0,1])

In [ ]:
plt.hist(cre, range=(-1., 1.), bins=16)
plt.title('Correlation between SLL and $C_l/C_d$', fontsize='large')
plt.xlabel('Pearson Correlation Coefficient')
plt.ylabel('Number of Trials')
plt.savefig('cor.svg')
plt.show()

In [ ]:
for i in range(6):
    print((np.array(cre) >= i*0.1).sum() / 250)

# Scatter with Normalization

In [ ]:
def normalize(x):
    mean = np.mean(x, axis=1, keepdims=True)
    std = np.std(x, axis=1, keepdims=True)
    return (x - mean) / std

In [ ]:
sll_n = normalize(ll_pred)
prf_n = normalize(rs)

In [ ]:
for i in range(250):
    plt.scatter(sll_n[i], prf_n[i], c='black', s=10, alpha=0.4)
plt.title('Standardized SLL vs Standardized $C_l/C_d$', fontsize='large')
plt.xlabel('Standardized Surrogate Log Likelihood') 
plt.ylabel('Standardized $C_l/C_d$') 
plt.plot([-3.5, 3.5],[-3.5, 3.5], 'r--')
plt.xlim(-3.5, 3.5)
plt.ylim(-3.5, 3.5)
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('scatter.svg')
plt.show()
